In [1]:
import sys
# sys.path.append('..')
import tree
import numpy as np
import os
from rte_dataset.builders import pipeline

In [2]:
DATA_PATH = "/workspaces/deeprte/data/raw/train/L"
DATA_NAME_LIST = ["L.mat"]

p = pipeline.DataPipeline(DATA_PATH, DATA_NAME_LIST)

data = p.process(normalization=True)
# data = p.process(normalization=False)

(1000, 40, 41)


/usr/local/lib/python3.11/site-packages/rte_dataset/builders/utils.py:70: RuntimeWarning: invalid value encountered in divide
  return (data - np.min(data)) / _range, _min, _range


In [3]:
tree.map_structure(lambda x: x.shape if isinstance(x, np.ndarray) else x, data)

{'functions': {'sigma': (1000, 40, 40, 2),
  'psi_label': (1000, 40, 40, 24),
  'scattering_kernel': (1000, 40, 40, 24, 24),
  'boundary_scattering_kernel': (1000, 160, 12, 24),
  'self_scattering_kernel': (1000, 24, 24),
  'boundary': (1000, 160, 12),
  'q': (1000, 40, 40)},
 'grid': {'position_coords': (40, 40, 2),
  'velocity_coords': (24, 2),
  'phase_coords': (40, 40, 24, 4),
  'boundary_coords': (160, 12, 4),
  'boundary_weights': (160, 12),
  'velocity_weights': (24,)},
 'shape': {'num_examples': 1000,
  'num_position_coords': 1600,
  'num_velocity_coords': 24,
  'num_phase_coords': 38400,
  'num_boundary_coords': np.int64(1920)},
 'normalization': {'psi_min': np.float32(0.0005234143),
  'psi_range': np.float32(1.2674637),
  'boundary_min': np.float32(0.0),
  'boundary_range': np.float32(0.0)}}

In [4]:
data["functions"]["sigma_a"] = data["functions"]["sigma"][...,-1]
data["functions"]["sigma_t"] = data["functions"]["sigma"][...,0]
data["functions"]["scattering_kernel"] = data["functions"]["self_scattering_kernel"]

# split = 0.8
# train_data = {}
# test_data = {}
# for key in data["functions"].keys():
#     train_data[key] = data["functions"][key][:int(split*data["functions"][key].shape[0])]
#     test_data[key] = data["functions"][key][int(split*data["functions"][key].shape[0]):]

# file_name = os.path.splitext(DATA_NAME_LIST[0])[0]
# train_file_path = os.path.join("../data/", file_name+"_train_normalized"+".npz")
# test_file_path = os.path.join("../data/", file_name+"_test_normalized"+".npz")
# np.savez(train_file_path, **train_data, phase_coords=data["grid"]["phase_coords"], weights=data["grid"]["velocity_weights"])
# np.savez(test_file_path, **test_data, phase_coords=data["grid"]["phase_coords"], weights=data["grid"]["velocity_weights"])

In [5]:
file_name = os.path.splitext(DATA_NAME_LIST[0])[0]
# train_file_path = os.path.join("../data/", file_name+"_train_normalized"+".npz")
file_path = os.path.join(DATA_PATH, file_name+".npz")

np.savez(file_path, **data["functions"], phase_coords=data["grid"]["phase_coords"], weights=data["grid"]["velocity_weights"])